In [1]:
%matplotlib notebook

from numpy import *
import numpy.linalg 
from numpy.random import rand
from matplotlib.pyplot import *


In [2]:
import subprocess
import shlex
import os

<hr style="border-width:4px; border-color:coral"></hr>

## Run Smoke3d

<hr style="border-width:4px; border-color:coral"></hr>

This notebook has two parts. 

* **Part 1:** Here, we create artificial data to simulate "observations" at gauges at a final output time $T$. These gauges are described to be observation stations where PM 2.5 concentrations are collected. In selecting these gauges, we used the United States Environmental Protection Agency Air Quality Data of 2020, to choose some cities in California that recorded high PM 2.5 trends. The final output time... 


* **Part 2:** Estimate the value of $S$ by solving a least squares problem.  This will be a linear or nonlinear  problem depending on the source model being solved, that is, a constant step function or Gaussian model, with dimension equal to the the number of gauges.  The function to be optimized involves a call to ForestClaw. 

As a first step, we test that we can run ForestClaw from this notebook.

### Problem setup

This code assumes that we will only observe solution at final time. 

In [3]:
# This is the command that will get called from the command line
## source_model = 1: constant function
## source_model = 2: gaussian function

source_model = 2  

S_true = 20
alpha_true = 0.0001

# Number of parameters to estimate
nparms = 2

shell_cmd = './latlong --user:source-model={:d} --user:S0={:.2f} --user:alpha={:.4e}'.format

!pwd

/Users/patriciaazike_1/projects/Smoke3d/code/Smoke3d/examples/latlong


## Test ForestClaw

<hr style="border-width:4px; border-color:coral"></hr>

Here we test ForestClaw on a small problem to ensure that the whole setup works. 

In [4]:
# mpirun command

cmd_test = './latlong --outstyle=3 --nout=10 --nstep=10'

#cmd_test = 'pwd'
arg_list = shlex.split(cmd_test) 
output = subprocess.run(arg_list)

if output.returncode != 0:
    print("latlong : Something bad happened!")
    print(output.stderr)
        
output

latlong : Something bad happened!
None


CompletedProcess(args=['./latlong', '--outstyle=3', '--nout=10', '--nstep=10'], returncode=143)

## Create Gauge file

<hr style="border-width:4px; border-color:coral"></hr>

Now, we create 9 guage files corresponding to data for 9 cities in California.

In [ ]:
m_gauges = 9

# Define gauges here in long/lat coordinates
#p_long = array([-120.5, -120, -119.5],dtype='float')
#p_lat = array([37]*m_gauges,dtype='float')

p_long = array([-119.0187, -119.8947, -120.625, -120.156, -119.375, -120.156, -119.667, -119.375, -120.1088],dtype='float')
p_lat = array([35.3210, 36.1389, 35.125, 34.5, 35.5938, 35.4375, 34.9429, 35.2812, 34.9681],dtype='float')


f = open('gauges.data','w')
f.write("{:d}\n".format(m_gauges))
t0 = 0
t1 = 1e10
for i in range(m_gauges):
    id = i
    f.write("{:5d} {:12.2f} {:12.2f} {:12.2e} {:12.2e}\n".format(id,p_long[i],p_lat[i],t0,t1))
f.close()

In [ ]:
# p_long = array([-119.0187, -119.8947, -120.625, -120.156, -119.375, -120.156, -119.667, -119.375, -120.1088],dtype='float')
# p_lat = array([35.3210, 36.1389, 35.125, 34.5, 35.5938, 35.4375, 34.9429, 35.2812, 34.9681],dtype='float')


In [ ]:
%%bash

cat gauges.data

## Create Data

<hr style="border-width:4px; border-color:coral"></hr>

Here we create some data.   This will read the gauge files created above and create output files `gauge00001.txt`, 
`gauge00002.txt`, ....

In [ ]:
from os.path import exists

# Read gauge files and collect data

def read_gauge_data():
    gauge_file = "gauge{:05d}.txt".format
    qvar_data = empty((m_gauges,1))

    errout = 0
    for i in range(m_gauges):
        gfile = gauge_file(i)
        if not exists(gfile):
            errout = 1
            return qvar_data,errout
        
        data = loadtxt(gfile)
        qvar_data[i] = data[-1,2]

    return qvar_data,errout

Here, we performed one run of the code with a $S_{true}$ value.  Then perturb the data to get our observations. We used a standard deviation of 0.15 to perturb the results we got from ForestClaw. 

## ForestClaw wrapper
<hr style="border-width:4px; border-color:coral"></hr> 

We can wrap ForestClaw in a Python function.   

* Reads data from `fclaw_options.ini`, spatial and temporal resolution

* A ForestClaw run will read data from a gauge file (created below) and report on values at gauges at regular time intervals, specified in ForetClaw options file `fclaw_options.ini'

* This wrapper takes two input parameters $S$ and $\alpha$ that control the source term.  The source term model is also used, but is set as a global variable above. 

* The output are gauge files, that are read in routines below.  The function $G(S,\alpha)$ will call this function and return the value of $q(x,t)$ at the final time, at each of the gauges.  


In [ ]:
# This calls ForestClaw

cmd = shell_cmd(source_model,S_true,alpha_true)   # Creates a command line call for any value of S

arg_list = shlex.split(cmd) 
output = subprocess.run(arg_list, stdout=subprocess.DEVNULL)

# Read gauges files that were created to get "observations"
qdata,errout = read_gauge_data()
if errout != 0:
    print("Problem reading gauge files {:s}".format(gfile))
    exit(0)

    
# TODO : Add a random perturbation to qvar_data vector entries

qvar_observations = qdata + 0.05 *rand(9,1)

print("Success!")
print("qvar_observations")
print("")
print(qvar_observations)


## Estimate $S_0$ from data

<hr style="border-width:4px; border-color:coral"></hr>

Think about linear least squares  to fit data to a polynomial : 

\begin{equation}
f(x)  = ax^2 + bx + c
\end{equation}

We set up a matrix equation for unknowns $(a,b,c)$, which we can store in a vector $\mathbf p = [a,b,c]^T$. 

The linear least squares problem is then to find

\begin{equation}
\overline{\mathbf p} = \mbox{argmin} \Vert V \mathbf p - \mathbf b \Vert^2
\end{equation}

where $V$ is a Vandermonde matrix and entries of $\mathbf b$ are values $y_j = f(x_j)$, $j = 1,2,\dots m$. 

More generally, we can solve 

\begin{equation}
\overline{S} = \mbox{argmin} \Vert \mathbf G(S)  - \mathbf d \Vert^2
\end{equation}

where $\mathbf G(S)$ represents the vector of values from $m$ gauges for parameter choice $S$. 


In [ ]:
# Some problem dependent parameters

# Function to being minimized. 
def F(p):    
    
    # If optimizing for S and alpha, p = [S,alpha]
    sval = p[0]
    aval = p[1]
    
    # print("sval = ",sval)
    cmd = shell_cmd(source_model,sval,aval)   # Creates a command line call for any value of S

    arg_list = shlex.split(cmd) 
    output = subprocess.run(arg_list, stdout=subprocess.DEVNULL)
    GS,errout = read_gauge_data()
    if errout !=0:
        print("Problem reading gauge files\n")
        exit(0)
    Fval = GS - qvar_observations
    Fval.resize(m_gauges)
    
    return Fval

### Compute Jacobian

In [ ]:

p = array([S_true,alpha_true])
hS = 1
halpha = 5e-5
h1v = array([hS,0],dtype='float')
h2v = array([0,halpha])

gs1 = F(p + h1v)
gs2 = F(p - h1v)
Jval_col1 = (gs1 - gs2)/(2*hS)
Jval_col1.resize((m_gauges,1))

# TODO : For source model 2, you have to include a second column for (F(alpha+h)- F(alpha-h))/(2*h)
if source_model == 2:
    gs1 = F(p + h2v)
    gs2 = F(p - h2v)
    Jval_col2 = (gs1 - gs2)/(2*halpha)
    Jval_col2.resize((m_gauges,1))
else:
    Jval_col2 = zeros((m_gauges,1))

Jval = hstack((Jval_col1, Jval_col2))

def jac(S):
    return Jval

In [ ]:
from scipy.optimize import least_squares

x0 = array([15,2.7e-4])
S_soln = least_squares(F,x0,jac=jac,method='lm',gtol=1e-12)
display(S_soln)
xsoln = S_soln.x
print("Solution is ",  S_soln.x)

## Rerun ForestClaw with estimated parameters and create output files

<hr style="border-width:4px; border-color:coral"></hr>

We now run ForestClaw one more time with estimated parameters.  This will create output files fort.q*

In [ ]:
# Create output files fort.*

resid = F(xsoln)
print("residual = \n",resid)

In [ ]:
cmd = shell_cmd(source_model,xsoln[0],xsoln[1])   # Creates a command line call for any value of S

arg_list = shlex.split(cmd) 
output = subprocess.run(arg_list, stdout=subprocess.DEVNULL)

# Read gauges files that were created to get "observations"
qest,errout = read_gauge_data()
if errout != 0:
    print("Problem reading gauge files {:s}".format(gfile))
    exit(0)

    
# TODO : Add a random perturbation to qvar_data vector entries

q_estimates = qest

print("Success!")
print("q_estimates = ")
print("")
print(q_estimates)

In [ ]:
figure(1)
clf
x = arange(1,10)
plot(x, qvar_observations, 'r*', label='obs')
plot(x, q_estimates, 'b*', label='est')
legend()
xlabel("Gauge")
ylabel("q($\mu gm^{-3}$)")
title("PM 2.5 Concentration")
show()

In [ ]:
figure(2)
clf
plot(x, resid, 'r*')
xlabel("Gauge")
ylabel("Residual")
title("Residuals")
show()

In [ ]:
chi_sq = linalg.norm(resid,2)
print("chi_sq = ", chi_sq)

In [ ]:
from scipy.stats import chisquare
chisquare(q_estimates)

In [ ]:
# from scipy import stats

# rvs1 = stats.norm.rvs(loc=5, scale=10, size=500, random_state=q_estimates)
